<a href="https://colab.research.google.com/github/Ryoo1014/Ryoo1014/blob/main/%E5%82%B5%E5%8B%99%E4%B8%8D%E5%B1%A5%E8%A1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from pandas.api.types import is_numeric_dtype
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.datasets import make_classification
from keras.models import Sequential
from keras.layers import Dense, Activation
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

In [3]:
def write_submission(id, pred, output_file):
    """予測結果を出力します."""
    df = pd.DataFrame({'id': id, 'price_range': pred})
    df.to_csv(output_file, index=False, header=False)

In [9]:
train_df = pd.read_csv('/content/債務不履行/train.csv')
test_df = pd.read_csv('/content/債務不履行/test.csv')

In [10]:
train_df

,id,loan_amnt,term,interest_rate,grade,employment_length,purpose,credit_score,application_type,loan_status
0,88194295,1800.0,3 years,14.49,C4,NaN,debt_consolidation,665.0,Individual,FullyPaid
1,5146039,1200.0,5 years,16.29,C4,2 years,debt_consolidation,700.0,Individual,ChargedOff
2,3095896,2000.0,5 years,21.98,E4,10 years,home_improvement,670.0,Individual,FullyPaid
3,88625044,1000.0,3 years,8.59,A5,4 years,debt_consolidation,710.0,Individual,FullyPaid
4,1178189,1500.0,3 years,13.99,C1,4 years,debt_consolidation,680.0,Individual,FullyPaid
...,...,...,...,...,...,...,...,...,...,...
242151,3218672,1500.0,3 years,10.16,B1,5 years,debt_consolidation,690.0,Individual,FullyPaid
242152,91332527,1930.0,3 years,12.74,C1,10 years,other,695.0,Individual,ChargedOff
242153,123604615,780.0,3 years,14.08,C3,10 years,debt_consolidation,730.0,Individual,FullyPaid
242154,2306469,472.5,3 years,19.72,D5,NaN,home_improvement,685.0,Individual,FullyPaid


In [11]:
train_df.info()
print('-'*40)
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 242156 entries, 0 to 242155
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   id                 242156 non-null  int64  
 1   loan_amnt          242156 non-null  float64
 2   term               242156 non-null  object 
 3   interest_rate      242156 non-null  float64
 4   grade              242156 non-null  object 
 5   employment_length  228971 non-null  object 
 6   purpose            242156 non-null  object 
 7   credit_score       242156 non-null  float64
 8   application_type   242156 non-null  object 
 9   loan_status        242156 non-null  object 
dtypes: float64(3), int64(1), object(6)
memory usage: 18.5+ MB
----------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26906 entries, 0 to 26905
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -

In [12]:
train_df.describe(include='all')

,id,loan_amnt,term,interest_rate,grade,employment_length,purpose,credit_score,application_type,loan_status
count,2.421560e+05,242156.000000,242156,242156.000000,242156,228971,242156,242156.000000,242156,242156
unique,NaN,NaN,2,NaN,35,11,13,NaN,2,2
top,NaN,NaN,3 years,NaN,B3,10 years,debt_consolidation,NaN,Individual,FullyPaid
freq,NaN,NaN,188699,NaN,16937,79095,141059,NaN,236086,193815
mean,5.642844e+07,1417.203456,NaN,13.988211,NaN,NaN,NaN,697.442331,NaN,NaN
std,4.790252e+07,872.780311,NaN,4.847934,NaN,NaN,NaN,31.922166,NaN,NaN
min,5.571600e+04,100.000000,NaN,5.320000,NaN,NaN,NaN,660.000000,NaN,NaN
25%,3.357649e+06,750.000000,NaN,10.640000,NaN,NaN,NaN,675.000000,NaN,NaN
50%,8.558081e+07,1200.000000,NaN,13.490000,NaN,NaN,NaN,690.000000,NaN,NaN
75%,9.245125e+07,2000.000000,NaN,16.990000,NaN,NaN,NaN,715.000000,NaN,NaN


In [13]:
train_df = train_df.drop({'id'}, axis=1)
Id = test_df['id']
train_objective = train_df['loan_status']
train_features = train_df.drop('loan_status', axis=1)
test_features = test_df.drop('id', axis=1)